In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql
!ln -s /var/run/postgresql/.s.PGSQL.5432 /tmp/.s.PGSQL.5432

In [2]:
conn = psycopg2.connect(database = "datamyne20170707", user = "ram22")
cur = conn.cursor()

In [3]:
#If we ever get around to load other fields more specifically, these should be loaded as text with no length limit
loadAsText = ["shipper_declared", "shipper_declared_address", "consignee_declared",
              "consignee_declared_address", "notify", "description", "marks",
              "master_shipper", "master_consignee_unified", "master_notify"]

In [5]:
inputFile = open('/home/ram22/dataDrive/dataProjects/MeshIntel Datamyne Delivery.csv', 'r')
#inputFile = open('MeshIntel Datamyne Delivery_n10.csv', 'r')
varNames = inputFile.readline()

# get rid of \n at end of line and text delimeter, then create list
varNames = varNames[:-2].replace('"', '').split(',')
print(varNames)

['bill_id', 'bill_of_lading_number', 'vessel_date', 'company_field_found', 'country_of_origin', 'world_region_by_country_of_origin', 'port_of_departure', 'country_by_port_of_departure', 'world_region_by_port_of_departure', 'us_port', 'us_port_state', 'us_coast', 'final_destination', 'final_destination_state_code', 'place_of_receipt', 'country_by_place_of_receipt', 'world_region_by_place_of_receipt', 'foreign_destination', 'shipper_unified_id', 'shipper_duns_number', 'shipper_unified', 'shipper_declared', 'shipper_declared_address', 'consignee_unified_id', 'consignee_duns_number', 'consignee_unified', 'consignee_declared', 'consignee_declared_address', 'consignee_declared_state', 'consignee_declared_state_abb', 'consignee_declared_city', 'consignee_declared_zip_code', 'consignee_declared_zip_code_state', 'consignee_declared_zip_code_county', 'consignee_declared_zip_code_city', 'consignee_declared_zip_code_latitude', 'consignee_declared_zip_code_longitude', 'notify', 'hs_code_2_digits', 

In [6]:
cur.execute("DROP TABLE IF EXISTS billoflading;")

i = 0
for field in varNames:
    if i == 0:
        cur.execute(sql.SQL("CREATE TABLE billoflading ({0} varchar);").format(sql.Identifier(field)))
        i += 1
    else:
        cur.execute(sql.SQL("ALTER TABLE billoflading ADD {0} varchar;").format(sql.Identifier(field)))

cur.copy_expert("COPY billoflading FROM STDIN WITH CSV HEADER", inputFile)


In [7]:
inputFile.close()
conn.commit()
conn.close()